# PRZYGOTOWANIE ZBIORU DANYCH

In [1]:
#biblioteki do przygotowania zbioru
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import xgboost, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import nltk
#
from textblob import TextBlob

import numpy as np

In [ ]:
#POBRANIE SŁÓW ZE STOP LISTY
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')

#pobierz Punkt Tokenizer Models
nltk.download('punkt')

#POBRANIE ANGIELSKIEJ BAZY DANYCH: rzeczowników, czasowników, przymiotników, przysłówków i zaimków
from textblob import Word
nltk.download('wordnet')

#POBRANIE Greedy Averaged Perceptron tagger
nltk.download('averaged_perceptron_tagger')
### ŁADOWANIE GŁÓWNEGO ZBIORU DO NAUKI

### ŁADOWANIE GŁÓWNEGO ZBIORU DO NAUKI


In [ ]:

trainSet=pd.read_csv("corona.csv",encoding='latin1')
testSet=pd.read_csv("corona-test.csv",encoding='latin1')
mainSet = pd.concat([trainSet,testSet])
mainSet

### ZWĘŻENIE OPINII W ZBIORZE 


In [2]:
mainSet["Sentiment"] = mainSet["Sentiment"].astype(str)
def classes_def(x):
    if x ==  "Extremely Positive":
        return "positive"
    elif x == "Extremely Negative":
        return "negative"
    elif x == "Negative":
        return "negative"
    elif x ==  "Positive":
        return "positive"
    else:
        return "neutral"
    
mainSet['Sentiment']=mainSet['Sentiment'].apply(lambda x:classes_def(x))
mainSet

In [ ]:
mainSet['text']=mainSet.OriginalTweet
mainSet['text']=mainSet['text'].astype(str)
sentiment = mainSet.groupby('Sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
sentiment=sentiment.set_index('Sentiment')
sentiment

In [ ]:
sentiment.plot(kind="bar", legend=None)

In [ ]:
#wyświetl 100 rekordów

print(mainSet.head(100))
print(mainSet.describe())

# ANALIZY ZBIORU

In [ ]:
mainSet['word_count']= mainSet['OriginalTweet'].apply(lambda x: len(str(x).split(" ")))
mainSet[['OriginalTweet', 'word_count']].head(10)

#### średnia długość słowa

In [ ]:
def avgWord(sentence):
    words=sentence.split()
    return (sum(len(word) for word in words)/len(words))


mainSet['avg_words'] = mainSet['OriginalTweet'].apply(lambda x:avgWord(x))
mainSet[['OriginalTweet', 'avg_words']].head(10)

### ilość znaków tweeta

In [ ]:
mainSet['char_count']=mainSet['OriginalTweet'].str.len()
mainSet[['OriginalTweet', 'char_count']].head(10)

### słowa ze stoplisty

In [ ]:
mainSet['stopwords']=mainSet['OriginalTweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
mainSet[['OriginalTweet', 'stopwords']].head(10)

### ZNAKI SPECJALNE W TWEET'CIE

In [ ]:
specialPatt=r"[^a-zA-Z0-9 ]"
mainSet['special_characters']=mainSet['OriginalTweet'].str.findall(specialPatt)
mainSet['special_characters']=mainSet['special_characters'].apply(lambda x: len(x))
mainSet[['OriginalTweet', 'special_characters']].head(10)
mainSet

### ILOŚĆ SŁÓW Z WIELKIEJ LITERY W TWEET'CIE

In [ ]:
mainSet['numerics'] = mainSet['OriginalTweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
mainSet[['OriginalTweet', 'numerics']].head(10)

In [ ]:
mainSet.corr()

# NORMALIZACJA ZBIORU

In [ ]:
# oryginalna kopia zbioru
mainSetDFRaw = mainSet.copy()


In [3]:
mainSet['OriginalTweet'] = mainSet['OriginalTweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
mainSet['OriginalTweet'].head(10)

NameError: name 'mainSet' is not defined

#### usunięcie znaków specjalnych

In [ ]:
mainSet['OriginalTweet'] = mainSet['OriginalTweet'].str.replace("[^\w\s]", '')
mainSet['OriginalTweet'].head(10)

#### usunięcie znaków

In [ ]:
mainSet['OriginalTweet'] = mainSet['OriginalTweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
mainSet['OriginalTweet'].head(10)

#### NAJCZĘSCIEJ WYSTĘPUJĄCE SŁOWA

In [ ]:
freq=pd.Series(' '.join(mainSet['OriginalTweet']).split()).value_counts()[:5]
freq

#### usunięcie najczejściej występujących słów

In [ ]:
freq=list(freq.index)
mainSet['OriginalTweet']= mainSet['OriginalTweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
mainSet['OriginalTweet'].head(10)

#### NAJRZADZIEJ WYSTĘPUJĄCE

In [4]:
leastFreq=pd.Series(' '.join(mainSet['OriginalTweet']).split()).value_counts()[-30:]
leastFreq

NameError: name 'pd' is not defined

#### usunięcie najrzadziej wystepujących słów

In [5]:
leastFreq= list(leastFreq.index)

mainSet['OriginalTweet'] = mainSet['OriginalTweet'].apply(lambda x: " ".join(x for x in x.split() if x not in leastFreq))


mainSet['OriginalTweet'].head(10)

NameError: name 'leastFreq' is not defined

#### LEMATYZACJA

In [6]:
mainSet['OriginalTweet'] = mainSet['OriginalTweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

mainSet['OriginalTweet'].head(10)

In [ ]:
text = pd.Series.to_string(mainSet['OriginalTweet'])
text

# WEKTORYZACJA

In [ ]:
mainSet['OriginalTweet'][1:2]

In [ ]:
# sprawdzenie wystąpień w 1 wierszu
tf1 = (mainSet['OriginalTweet'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

In [ ]:
for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(mainSet.shape[0]/(len(mainSet[mainSet['OriginalTweet'].str.contains(word)])))
tf1

In [ ]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:

tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(mainSet['OriginalTweet'])


In [ ]:
train_vect